**GOD BLESS**

In [1]:
import pandas as pd
import numpy as np
import statistics
from sklearn.preprocessing import StandardScaler, LabelEncoder
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score as cvs
import warnings

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.info()
train.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11153 entries, 0 to 11152
Data columns (total 22 columns):
job_level                                11153 non-null object
job_duration_in_current_job_level        11153 non-null float64
person_level                             11153 non-null object
job_duration_in_current_person_level     11153 non-null float64
job_duration_in_current_branch           11153 non-null float64
Employee_type                            11153 non-null object
gender                                   11153 non-null int64
age                                      11153 non-null int64
marital_status_maried(Y/N)               11153 non-null object
number_of_dependences                    11153 non-null int64
Education_level                          11153 non-null object
GPA                                      11153 non-null float64
year_graduated                           11153 non-null int64
job_duration_from_training               11153 non-null int64
branch_rot

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,age,marital_status_maried(Y/N),number_of_dependences,...,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal,Best Performance
0,JG04,1.352775,PG03,1.352775,1.732051,RM_type_A,2,1985,Y,1,...,2011,4,2,3,2,3,0,46.37,0.0,0
1,JG04,1.292285,PG03,1.292285,1.039230,RM_type_A,2,1989,Y,0,...,2007,4,4,3,0,0,1,47.68,0.0,0
2,JG05,2.565151,PG06,2.308679,1.780449,RM_type_C,1,1970,Y,1,...,1989,28,10,9,6,2,1,40.73,0.0,0
3,JG05,2.828427,PG06,1.000000,1.385641,RM_type_A,2,1967,Y,2,...,1987,29,11,6,0,4,10,47.42,0.0,1
4,JG05,2.828427,PG06,2.828427,0.707107,RM_type_A,2,1965,Y,3,...,1985,30,9,8,2,3,0,47.18,0.0,0
5,JG05,2.828427,PG06,2.121320,1.118034,RM_type_A,1,1971,Y,3,...,1990,28,7,9,4,0,0,48.46,0.0,0
6,JG05,2.828427,PG06,2.345208,2.102380,RM_type_A,1,1969,Y,2,...,1988,29,8,6,2,8,0,60.38,0.0,0
7,JG05,2.828427,PG06,2.236068,1.581139,RM_type_C,2,1968,Y,2,...,1987,29,6,8,1,3,4,14.71,0.0,0
8,JG04,1.118034,PG03,1.118034,0.412311,RM_type_B,1,1989,Y,0,...,2007,3,2,2,0,9,0,26.76,0.0,0
9,JG04,1.118034,PG03,1.118034,0.412311,RM_type_B,2,1991,N,0,...,2009,3,2,2,0,3,0,41.45,0.0,1


In [4]:
test.info()
test.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 21 columns):
job_level                                6000 non-null object
job_duration_in_current_job_level        6000 non-null float64
person_level                             6000 non-null object
job_duration_in_current_person_level     6000 non-null float64
job_duration_in_current_branch           6000 non-null float64
Employee_type                            6000 non-null object
gender                                   6000 non-null int64
age                                      6000 non-null int64
marital_status_maried(Y/N)               6000 non-null object
number_of_dependences                    6000 non-null int64
Education_level                          6000 non-null object
GPA                                      6000 non-null float64
year_graduated                           6000 non-null int64
job_duration_from_training               6000 non-null int64
branch_rotation           

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,age,marital_status_maried(Y/N),number_of_dependences,...,GPA,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal
0,JG04,1.256981,PG03,1.256981,0.707107,RM_type_A,1,1988,N,0,...,3.08,2009,6,4,2,6,3,7,84.02,0.0
1,JG04,1.224745,PG03,1.224745,1.256981,RM_type_B,2,1991,N,0,...,3.31,2014,2,2,2,0,1,0,63.74,0.0
2,JG04,0.500000,PG03,0.500000,1.081665,RM_type_A,2,1989,Y,1,...,3.44,2011,4,2,2,0,2,1,88.19,1.0
3,JG04,1.256981,PG03,1.256981,1.802776,RM_type_A,2,1990,Y,1,...,0.00,2014,3,1,3,0,3,0,25.94,0.0
4,JG04,1.352775,PG03,1.352775,1.224745,RM_type_B,1,1985,Y,0,...,3.34,2007,5,3,2,0,3,0,82.32,0.0
5,JG04,1.081665,PG03,1.081665,1.581139,RM_type_B,2,1990,Y,0,...,3.33,2011,6,5,4,0,3,0,72.10,0.0
6,JG04,2.723968,PG04,2.000000,1.385641,RM_type_A,2,1982,Y,2,...,3.07,2004,7,3,4,3,0,1,35.58,0.0
7,JG04,1.352775,PG03,1.352775,1.322876,RM_type_A,2,1983,Y,3,...,2.49,2007,5,3,4,0,0,0,26.49,0.0
8,JG04,1.256981,PG03,1.256981,0.707107,RM_type_B,2,1988,Y,0,...,0.00,2012,4,3,2,0,5,4,68.09,0.0
9,JG04,1.118034,PG03,1.118034,1.581139,RM_type_B,2,1993,N,0,...,0.00,2015,3,1,2,0,2,0,70.53,0.0


In [5]:
#set age
train['age'] = 2021 - train['age']
test['age'] = 2021 - test['age']
train.loc[0:10, "age"]

0     36
1     32
2     51
3     54
4     56
5     50
6     52
7     53
8     32
9     30
10    55
Name: age, dtype: int64

In [6]:
train["year_graduated"] = 2021 - train["year_graduated"]
test["year_graduated"] = 2021 - test["year_graduated"]
train["year_graduated"].head(25)

0     10
1     14
2     32
3     34
4     36
5     31
6     33
7     34
8     14
9     12
10    36
11    14
12    10
13    22
14    29
15    12
16    13
17    12
18     9
19     8
20    32
21    12
22    33
23    38
24    12
Name: year_graduated, dtype: int64

In [7]:
#Fill Last_achievement_%
la_mean = train["Last_achievement_%"].mean()
la_median = train["Last_achievement_%"].median()
la_mode = statistics.mode(train["Last_achievement_%"].dropna())
print(la_mean, la_median, la_mode)
train["Last_achievement_%"] = train["Last_achievement_%"].replace(np.NaN, la_mean)

72.23490225968477 71.67500000000001 13.0


In [8]:
train["Last_achievement_%"].isnull().sum()

0

In [9]:
#Fill Achievement_above_100%_during3quartal
aa_mean = train["Achievement_above_100%_during3quartal"].mean()
aa_median = train["Achievement_above_100%_during3quartal"].median()
aa_mode = statistics.mode(train["Achievement_above_100%_during3quartal"].dropna())
print(aa_mean, aa_median, aa_mode)
train["Achievement_above_100%_during3quartal"] = train["Achievement_above_100%_during3quartal"].replace(np.NaN, aa_median)

0.679071018651363 0.0 0.0


In [10]:
train["Achievement_above_100%_during3quartal"].isnull().sum()

0

In [11]:
train.drop('GPA', axis = 1)

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,age,marital_status_maried(Y/N),number_of_dependences,...,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal,Best Performance
0,JG04,1.352775,PG03,1.352775,1.732051,RM_type_A,2,36,Y,1,...,10,4,2,3,2,3,0,46.37,0.0,0
1,JG04,1.292285,PG03,1.292285,1.039230,RM_type_A,2,32,Y,0,...,14,4,4,3,0,0,1,47.68,0.0,0
2,JG05,2.565151,PG06,2.308679,1.780449,RM_type_C,1,51,Y,1,...,32,28,10,9,6,2,1,40.73,0.0,0
3,JG05,2.828427,PG06,1.000000,1.385641,RM_type_A,2,54,Y,2,...,34,29,11,6,0,4,10,47.42,0.0,1
4,JG05,2.828427,PG06,2.828427,0.707107,RM_type_A,2,56,Y,3,...,36,30,9,8,2,3,0,47.18,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11148,JG04,1.256981,PG03,1.256981,1.224745,RM_type_A,2,35,Y,1,...,5,4,3,3,0,4,1,50.87,0.0,0
11149,JG04,1.385641,PG03,1.385641,1.802776,RM_type_B,1,33,Y,2,...,6,3,1,2,0,6,0,79.91,0.0,0
11150,JG04,1.256981,PG03,1.256981,1.385641,RM_type_B,2,30,Y,0,...,4,2,1,2,0,4,0,78.38,0.0,0
11151,JG04,1.118034,PG03,1.118034,0.000000,RM_type_A,1,34,Y,1,...,9,5,5,4,0,4,1,85.94,1.0,0


In [12]:
#Object data type treatment
le = LabelEncoder()
#every cols
obj_cols = ['job_level', 'person_level', 'Employee_type', 'marital_status_maried(Y/N)', 'Education_level']
for col in obj_cols:
    labels = train[col].unique().tolist() + test[col].unique().tolist()
    le.fit(labels)
    test[col] = le.transform(test[col].astype(str))
    train[col] = le.transform(train[col].astype(str))
    print(list(le.classes_))

['JG03', 'JG04', 'JG05', 'JG06']
['PG01', 'PG02', 'PG03', 'PG04', 'PG05', 'PG06', 'PG07', 'PG08']
['RM_type_A', 'RM_type_B', 'RM_type_C']
['N', 'Y']
['level_0', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5']


In [13]:
train

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,age,marital_status_maried(Y/N),number_of_dependences,...,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal,Best Performance
0,1,1.352775,2,1.352775,1.732051,0,2,36,1,1,...,10,4,2,3,2,3,0,46.37,0.0,0
1,1,1.292285,2,1.292285,1.039230,0,2,32,1,0,...,14,4,4,3,0,0,1,47.68,0.0,0
2,2,2.565151,5,2.308679,1.780449,2,1,51,1,1,...,32,28,10,9,6,2,1,40.73,0.0,0
3,2,2.828427,5,1.000000,1.385641,0,2,54,1,2,...,34,29,11,6,0,4,10,47.42,0.0,1
4,2,2.828427,5,2.828427,0.707107,0,2,56,1,3,...,36,30,9,8,2,3,0,47.18,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11148,1,1.256981,2,1.256981,1.224745,0,2,35,1,1,...,5,4,3,3,0,4,1,50.87,0.0,0
11149,1,1.385641,2,1.385641,1.802776,1,1,33,1,2,...,6,3,1,2,0,6,0,79.91,0.0,0
11150,1,1.256981,2,1.256981,1.385641,1,2,30,1,0,...,4,2,1,2,0,4,0,78.38,0.0,0
11151,1,1.118034,2,1.118034,0.000000,0,1,34,1,1,...,9,5,5,4,0,4,1,85.94,1.0,0


In [14]:
test

,job_level,job_duration_in_current_job_level,person_level,job_duration_in_current_person_level,job_duration_in_current_branch,Employee_type,gender,age,marital_status_maried(Y/N),number_of_dependences,...,GPA,year_graduated,job_duration_from_training,branch_rotation,job_rotation,assign_of_otherposition,annual leave,sick_leaves,Last_achievement_%,Achievement_above_100%_during3quartal
0,1,1.256981,2,1.256981,0.707107,0,1,33,0,0,...,3.08,12,6,4,2,6,3,7,84.02,0.0
1,1,1.224745,2,1.224745,1.256981,1,2,30,0,0,...,3.31,7,2,2,2,0,1,0,63.74,0.0
2,1,0.500000,2,0.500000,1.081665,0,2,32,1,1,...,3.44,10,4,2,2,0,2,1,88.19,1.0
3,1,1.256981,2,1.256981,1.802776,0,2,31,1,1,...,0.00,7,3,1,3,0,3,0,25.94,0.0
4,1,1.352775,2,1.352775,1.224745,1,1,36,1,0,...,3.34,14,5,3,2,0,3,0,82.32,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,1,1.292285,2,1.292285,0.707107,1,2,33,1,0,...,3.01,6,2,2,2,0,4,0,58.60,0.0
5996,1,1.352775,2,1.352775,1.581139,0,2,32,1,1,...,3.56,10,5,3,2,7,3,0,90.64,3.0
5997,1,1.385641,2,1.385641,1.224745,0,2,30,1,0,...,3.59,8,3,2,3,0,16,0,13.02,0.0
5998,2,1.385641,4,1.385641,0.648074,0,2,41,1,2,...,2.93,18,14,7,6,8,2,0,82.26,0.0


In [15]:
X_train = train.drop('Best Performance', axis = 1)
y_train = train['Best Performance']
X_train_, X_test_, y_train_, y_test_ = train_test_split(X_train, y_train, test_size = 0.2, random_state = 69, stratify = y_train)

In [16]:
class lgbm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'boosting_type':'gbdt', 'class_weight':None, 'colsample_bytree':1.0, 
                  'importance_type':'split', 'learning_rate':0.1,
                  'min_child_samples':20, 'min_split_gain':0.0, 'n_estimators':100, 'objective':None,
                  'random_state':0, 'reg_alpha':0.0, 'reg_lambda':0.0, 'silent':True,
                  'subsample':1.0, 'subsample_for_bin':200000, 'subsample_freq':0}
        params['num_leaves'] = int(param['num_leaves'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_depth'] = int(param['max_depth'])
        params['learning_rate'] = 0.1
        params['min_data_in_bin'] = 1
        params['min_data'] = 1
        
        params['min_child_samples'] = int(param['min_child_samples'])
        params['bagging_fraction'] = param['bagging_fraction']
        params['lambda_l1'] = param['lambda_l1']
        params['lambda_l2'] = param['lambda_l2']

        return params
        
    def evaluate(self, min_child_weight, max_depth, num_leaves,
                min_child_samples, bagging_fraction, lambda_l1, lambda_l2):
        params = {'num_leaves':num_leaves, 
                  'min_child_weight':min_child_weight, 
                  'max_depth':max_depth,
                  'min_child_samples':min_child_samples,
                  'bagging_fraction' : bagging_fraction,
                  'lambda_l1' : lambda_l1,
                  'lambda_l2' : lambda_l2}
        
        params = self.clean_param(params)

        lgbm_model = lgb.LGBMClassifier(**params)
        lgbm_model.fit(self.x_train, self.y_train)
        y_pred = lgbm_model.predict_proba(self.x_test)
        predictions = y_pred[:,1]
#         rmse = np.sqrt(mean_squared_error(self.y_test, predictions))
#         return -1*rmse
        acc = roc_auc_score(self.y_test,predictions)
        return acc

lt = lgbm_target(X_train_, y_train_, X_test_, y_test_)
lgbmBO = BayesianOptimization(lt.evaluate, {'min_child_weight': (1, 100),
                                            'max_depth': (1, 100),
                                            'num_leaves': (10, 100),
                                            'min_child_samples' :(1,100),
                                            'bagging_fraction' : (0.5,1),
                                            'lambda_l1' : (0,1),
                                            'lambda_l2' : (0,1)
                                           }, 
                             random_state=23)

lgbmBO.maximize(init_points=23, n_iter=100)
warnings.filterwarnings('ignore')

|   iter    |  target   | baggin... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=22, will be overridden by min_data=1. Current value: min_data_in_leaf=1
[LightGBM] [Warning] lambda_l1 is set=0.9469626038148141, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9469626038148141
[LightGBM] [Warning] bagging_fraction is set=0.7586489419232947, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7586489419232947
[LightGBM] [Warning] lambda_l2 is set=0.7654597593969069, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.7654597593969069
|  1        |  0.5925   |  0.7586   |  0.947    |  0.7655   |  28.96    |  22.88    |  68.94    |  25.04    |
[LightGBM] [Warning] min_data_in_leaf is set with min_child_samples=88, will be overridden by min_data=1. Current va

In [17]:
lgbmBO.max['params']

{'bagging_fraction': 0.7947909326073572,
 'lambda_l1': 0.9784269160148309,
 'lambda_l2': 0.8450938221263111,
 'max_depth': 7.442468466662155,
 'min_child_samples': 30.179702007019195,
 'min_child_weight': 29.505509655757393,
 'num_leaves': 84.02197051930898}

In [18]:
params = {'bagging_fraction': 0.7947909326073572,
 'lambda_l1': 0.9784269160148309,
 'lambda_l2': 0.8450938221263111,
 'max_depth': 8,
 'min_child_samples': 30,
 'min_child_weight': 29.505509655757393,
 'num_leaves': 84}
model_lgb = lgb.LGBMClassifier(**params)
model_lgb.fit(X_train,y_train)
pred = model_lgb.predict_proba(test)[:,1]

[LightGBM] [Warning] lambda_l1 is set=0.9784269160148309, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.9784269160148309
[LightGBM] [Warning] bagging_fraction is set=0.7947909326073572, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7947909326073572
[LightGBM] [Warning] lambda_l2 is set=0.8450938221263111, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.8450938221263111


In [18]:
#print(y_pred)
#print(score)

In [19]:
#pred = lgbm.predict_proba(test)

In [19]:
print(pred)

[0.14175961 0.11157377 0.40718276 ... 0.26404956 0.17900321 0.12792001]


In [20]:
submission = pd.read_csv("sample_submission.csv")
submission['Best Performance'] = pred
submission

,index,Best Performance
0,0,0.141760
1,1,0.111574
2,2,0.407183
3,3,0.116215
4,4,0.129501
...,...,...
5995,5995,0.163024
5996,5996,0.082997
5997,5997,0.264050
5998,5998,0.179003


In [21]:
submission.to_csv('submissionLGBMBO-GPA.csv', index = False)